In [6]:
import requests

def new_req(endpoint, payload=None):
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': 'NcFrgbbBbhbTNLTFAHmmCcnqsmItgfOZ'
        },
      params=payload
    )

In [9]:
import datetime as dt
from IPython import display

start = dt.date(2025,8,1)
end = dt.date(2025,8,10)

results = []

while start < end:
    display.clear_output(wait=True)
    display.display(f'gathering data for {str(start)}')

    response = new_req(
        endpoint = 'data',
        payload = {
          'datasetid': 'GHCND',
          'locationid': 'CITY:US360019',
          'startdate': start,
          'enddate': start,
          'units': 'metric',
          'limit': '1000'
      }
    )

    if response.ok:
        results.extend(response.json().get('results', []))
        start += dt.timedelta(days=1)

'gathering data for 2025-08-09'

In [10]:
import pandas as pd
df = pd.DataFrame(results)
df.head()

,date,datatype,station,attributes,value
0,2025-08-01T00:00:00,PRCP,GHCND:US1NJBG0003,",,N,0730",8.1
1,2025-08-01T00:00:00,PRCP,GHCND:US1NJBG0015,",,N,0900",10.9
2,2025-08-01T00:00:00,PRCP,GHCND:US1NJBG0017,",,N,0715",15.0
3,2025-08-01T00:00:00,PRCP,GHCND:US1NJBG0018,",,N,0900",8.6
4,2025-08-01T00:00:00,SNOW,GHCND:US1NJBG0018,",,N,0900",0.0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2305 entries, 0 to 2304
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        2305 non-null   object 
 1   datatype    2305 non-null   object 
 2   station     2305 non-null   object 
 3   attributes  2305 non-null   object 
 4   value       2305 non-null   float64
dtypes: float64(1), object(4)
memory usage: 90.2+ KB


In [12]:
df.to_csv('weather_info_NT.csv', index=False)

In [13]:
from sqlite3.dbapi2 import connect
import sqlite3
with sqlite3.connect('weather.db') as connection:
  df.to_sql(
      'weather', connection, index=False, if_exists='replace'
  )

In [15]:
with sqlite3.connect('weather.db') as connection:
  weather = pd.read_sql('SELECT * FROM weather', connection)
weather.tail()

,date,datatype,station,attributes,value
2300,2025-08-09T00:00:00,TMIN,GHCND:USW00094789,",,W,2400",14.4
2301,2025-08-09T00:00:00,WDF2,GHCND:USW00094789,",,W,",210.0
2302,2025-08-09T00:00:00,WDF5,GHCND:USW00094789,",,W,",250.0
2303,2025-08-09T00:00:00,WSF2,GHCND:USW00094789,",,W,",6.3
2304,2025-08-09T00:00:00,WSF5,GHCND:USW00094789,",,W,",8.9


In [16]:
response = new_req(
    'stations',
    {
          'datasetid':'GHCND',
          'locationid':'CITY:US360019',
          'limit':1000
    }
)

In [18]:
stations=pd.DataFrame(response.json()['results'])[['id', 'name', 'latitude', 'longitude', 'elevation']]
stations.to_csv('weather_stations.csv', index=False)


In [19]:
import sqlite3
with sqlite3.connect('weather.db') as connection:
  df.to_sql(
      'stations', connection, index=False, if_exists='replace'
  )

In [20]:
#pandas

In [21]:
df = pd.read_csv('earthquakes.csv')

In [3]:
df.head()

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
0,NaN,NaN,37389218,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.008693,NaN,85.0,",ci37389218,",1.35,ml,...,",ci,",automatic,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventp...
1,NaN,NaN,37389202,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.020030,NaN,79.0,",ci37389202,",1.29,ml,...,",ci,",automatic,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475253925,https://earthquake.usgs.gov/earthquakes/eventp...
2,NaN,4.4,37389194,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.021370,28.0,21.0,",ci37389194,",3.42,ml,...,",ci,",automatic,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,earthquake,",dyfi,focal-mechanism,geoserve,nearby-cities,o...",-480.0,1539536756176,https://earthquake.usgs.gov/earthquakes/eventp...
3,NaN,NaN,37389186,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.026180,NaN,39.0,",ci37389186,",0.44,ml,...,",ci,",automatic,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475196167,https://earthquake.usgs.gov/earthquakes/eventp...
4,NaN,NaN,73096941,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.077990,NaN,192.0,",nc73096941,",2.16,md,...,",nc,",automatic,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,scit...",-480.0,1539477547926,https://earthquake.usgs.gov/earthquakes/eventp...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9332 entries, 0 to 9331
Data columns (total 26 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alert    59 non-null     object 
 1   cdi      329 non-null    float64
 2   code     9332 non-null   object 
 3   detail   9332 non-null   object 
 4   dmin     6139 non-null   float64
 5   felt     329 non-null    float64
 6   gap      6164 non-null   float64
 7   ids      9332 non-null   object 
 8   mag      9331 non-null   float64
 9   magType  9331 non-null   object 
 10  mmi      93 non-null     float64
 11  net      9332 non-null   object 
 12  nst      5364 non-null   float64
 13  place    9332 non-null   object 
 14  rms      9332 non-null   float64
 15  sig      9332 non-null   int64  
 16  sources  9332 non-null   object 
 17  status   9332 non-null   object 
 18  time     9332 non-null   int64  
 19  title    9332 non-null   object 
 20  tsunami  9332 non-null   int64  
 21  type     9332 

In [22]:
df.mag

0       1.35
1       1.29
2       3.42
3       0.44
4       2.16
        ... 
9327    0.62
9328    1.00
9329    2.40
9330    1.10
9331    0.66
Name: mag, Length: 9332, dtype: float64

In [23]:
df['mag']

0       1.35
1       1.29
2       3.42
3       0.44
4       2.16
        ... 
9327    0.62
9328    1.00
9329    2.40
9330    1.10
9331    0.66
Name: mag, Length: 9332, dtype: float64

In [25]:
df[['title', 'mag']]

,title,mag
0,"M 1.4 - 9km NE of Aguanga, CA",1.35
1,"M 1.3 - 9km NE of Aguanga, CA",1.29
2,"M 3.4 - 8km NE of Aguanga, CA",3.42
3,"M 0.4 - 9km NE of Aguanga, CA",0.44
4,"M 2.2 - 10km NW of Avenal, CA",2.16
...,...,...
9327,"M 0.6 - 9km ENE of Mammoth Lakes, CA",0.62
9328,"M 1.0 - 3km W of Julian, CA",1.00
9329,"M 2.4 - 35km NNE of Hatillo, Puerto Rico",2.40
9330,"M 1.1 - 9km NE of Aguanga, CA",1.10


In [30]:
df[[col for col in df.columns if col.startswith('mag')]]

,mag,magType
0,1.35,ml
1,1.29,ml
2,3.42,ml
3,0.44,ml
4,2.16,md
...,...,...
9327,0.62,md
9328,1.00,ml
9329,2.40,md
9330,1.10,ml


In [31]:
df[200:202]

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
200,NaN,NaN,73096626,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.01014,NaN,150.00,",nc73096626,",0.87,md,...,",nc,",automatic,1539398044050,"M 0.9 - 6km NW of The Geysers, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,scit...",-480.0,1539401522356,https://earthquake.usgs.gov/earthquakes/eventp...
201,NaN,NaN,00660890,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.04500,NaN,91.86,",nn00660890,",-0.10,ml,...,",nn,",reviewed,1539397417213,"M -0.1 - 37km ENE of Beatty, Nevada",0,earthquake,",geoserve,origin,phase-data,",-480.0,1539482709665,https://earthquake.usgs.gov/earthquakes/eventp...


In [35]:
df[200:202][['alert', 'cdi', 'code']]

,alert,cdi,code
200,NaN,NaN,73096626
201,NaN,NaN,00660890


In [36]:
df[['alert', 'cdi', 'code']]

,alert,cdi,code
0,NaN,NaN,37389218
1,NaN,NaN,37389202
2,NaN,4.4,37389194
3,NaN,NaN,37389186
4,NaN,NaN,73096941
...,...,...,...
9327,NaN,NaN,73086771
9328,NaN,NaN,38063967
9329,NaN,NaN,2018261000
9330,NaN,NaN,38063959


In [37]:
df[:][['alert', 'cdi', 'code']]


,alert,cdi,code
0,NaN,NaN,37389218
1,NaN,NaN,37389202
2,NaN,4.4,37389194
3,NaN,NaN,37389186
4,NaN,NaN,73096941
...,...,...,...
9327,NaN,NaN,73086771
9328,NaN,NaN,38063967
9329,NaN,NaN,2018261000
9330,NaN,NaN,38063959


In [43]:
df[['alert', 'cdi', 'code']][:] == df[:][['alert', 'cdi', 'code']]


,alert,cdi,code
0,False,False,True
1,False,False,True
2,False,True,True
3,False,False,True
4,False,False,True
...,...,...,...
9327,False,False,True
9328,False,False,True
9329,False,False,True
9330,False,False,True


In [44]:
df[['alert', 'cdi', 'code']][0:10]==df[0:10][['alert', 'cdi', 'code']]

,alert,cdi,code
0,False,False,True
1,False,False,True
2,False,True,True
3,False,False,True
4,False,False,True
5,False,False,True
6,False,False,True
7,False,False,True
8,False,False,True
9,False,False,True


In [45]:
df[['alert', 'cdi', 'code']][0:10].equals(df[['alert', 'cdi', 'code']][0:10])

True

In [46]:
df[['alert', 'cdi', 'code']][0:10].equals(df[0:10][['alert', 'cdi', 'code']])


True

In [47]:
df.loc[0:10, ['alert', 'cdi', 'code']]

,alert,cdi,code
0,NaN,NaN,37389218
1,NaN,NaN,37389202
2,NaN,4.4,37389194
3,NaN,NaN,37389186
4,NaN,NaN,73096941
5,NaN,NaN,2018286011
6,NaN,NaN,20280432
7,NaN,NaN,73096936
8,NaN,NaN,73096931
9,NaN,NaN,1000hbtn


In [48]:
df.columns

Index(['alert', 'cdi', 'code', 'detail', 'dmin', 'felt', 'gap', 'ids', 'mag',
       'magType', 'mmi', 'net', 'nst', 'place', 'rms', 'sig', 'sources',
       'status', 'time', 'title', 'tsunami', 'type', 'types', 'tz', 'updated',
       'url'],
      dtype='object')

In [49]:
df.columns.sort_values()

Index(['alert', 'cdi', 'code', 'detail', 'dmin', 'felt', 'gap', 'ids', 'mag',
       'magType', 'mmi', 'net', 'nst', 'place', 'rms', 'sig', 'sources',
       'status', 'time', 'title', 'tsunami', 'type', 'types', 'tz', 'updated',
       'url'],
      dtype='object')

In [51]:
df.iloc[0:10, [1,3]]

,cdi,detail
0,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...
1,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...
2,4.4,https://earthquake.usgs.gov/fdsnws/event/1/que...
3,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...
4,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...
5,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...
6,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...
7,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...
8,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...
9,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...


In [52]:
df.iloc[0:10, [1,3,5,7]]


,cdi,detail,felt,ids
0,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,",ci37389218,"
1,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,",ci37389202,"
2,4.4,https://earthquake.usgs.gov/fdsnws/event/1/que...,28.0,",ci37389194,"
3,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,",ci37389186,"
4,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,",nc73096941,"
5,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,",pr2018286011,"
6,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,",ak20280432,"
7,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,",nc73096936,"
8,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,",nc73096931,"
9,NaN,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,",us1000hbtn,"


In [53]:
df.iloc[0:10, [1-3]]


,updated
0,1539475395144
1,1539475253925
2,1539536756176
3,1539475196167
4,1539477547926
5,1539500579236
6,1539473596465
7,1539476642808
8,1539475027632
9,1539473712040


In [54]:
df.iloc[0:10, [-2]]


,updated
0,1539475395144
1,1539475253925
2,1539536756176
3,1539475196167
4,1539477547926
5,1539500579236
6,1539473596465
7,1539476642808
8,1539475027632
9,1539473712040


In [59]:
df.iloc[0:10,-4:]

,types,tz,updated,url
0,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventp...
1,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475253925,https://earthquake.usgs.gov/earthquakes/eventp...
2,",dyfi,focal-mechanism,geoserve,nearby-cities,o...",-480.0,1539536756176,https://earthquake.usgs.gov/earthquakes/eventp...
3,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475196167,https://earthquake.usgs.gov/earthquakes/eventp...
4,",geoserve,nearby-cities,origin,phase-data,scit...",-480.0,1539477547926,https://earthquake.usgs.gov/earthquakes/eventp...
5,",geoserve,origin,phase-data,",-300.0,1539500579236,https://earthquake.usgs.gov/earthquakes/eventp...
6,",geoserve,origin,",-540.0,1539473596465,https://earthquake.usgs.gov/earthquakes/eventp...
7,",geoserve,nearby-cities,origin,phase-data,scit...",-480.0,1539476642808,https://earthquake.usgs.gov/earthquakes/eventp...
8,",geoserve,nearby-cities,origin,phase-data,scit...",-480.0,1539475027632,https://earthquake.usgs.gov/earthquakes/eventp...
9,",geoserve,origin,phase-data,",540.0,1539473712040,https://earthquake.usgs.gov/earthquakes/eventp...


In [60]:
df.iloc[10:15, 6:10].equals(
    df.loc[10:14, 'gap':'magType'])

True

In [61]:
df.iloc[10:15, 6:10].equals(
    df.loc[10:14, 'detail':'mmi']
)

False

In [65]:
df.at[10, 'mag']

np.float64(0.5)

In [67]:
df.iat[10,10]

np.float64(nan)

In [71]:
df['mag']>2.5

0       False
1       False
2        True
3       False
4       False
        ...  
9327    False
9328    False
9329    False
9330    False
9331    False
Name: mag, Length: 9332, dtype: bool

In [76]:
len(df[df['mag']>=5])

151

In [77]:
len(df[df['mag']>=6])


12

In [78]:
len(df[df['mag']>=7])


2

In [79]:
len(df[df['mag']>=8])


0

In [83]:
df[df['mag']>=7]

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
837,green,4.1,1000haa3,https://earthquake.usgs.gov/fdsnws/event/1/que...,1.763,3.0,14.0,",us1000haa3,pt18283003,at00pgehsk,",7.0,mww,...,",us,pt,at,",reviewed,1539204500290,"M 7.0 - 117km E of Kimbe, Papua New Guinea",1,earthquake,",dyfi,finite-fault,general-text,geoserve,groun...",600.0,1539378744253,https://earthquake.usgs.gov/earthquakes/eventp...
5263,red,8.4,1000h3p4,https://earthquake.usgs.gov/fdsnws/event/1/que...,1.589,18.0,27.0,",us1000h3p4,us1000h4p4,",7.5,mww,...,",us,us,",reviewed,1538128963480,"M 7.5 - 78km N of Palu, Indonesia",1,earthquake,",dyfi,finite-fault,general-text,geoserve,groun...",480.0,1539123134531,https://earthquake.usgs.gov/earthquakes/eventp...


In [84]:
df.loc[df.mag>=7, ['alert', 'mag', 'magType', 'title', 'tsunami']]

,alert,mag,magType,title,tsunami
837,green,7.0,mww,"M 7.0 - 117km E of Kimbe, Papua New Guinea",1
5263,red,7.5,mww,"M 7.5 - 78km N of Palu, Indonesia",1


In [87]:
df.loc[(df['tsunami']==1)&(df['alert']=='red'), 
       ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']]

,alert,mag,magType,title,tsunami,type
5263,red,7.5,mww,"M 7.5 - 78km N of Palu, Indonesia",1,earthquake


In [ ]:
data = df.loc[(df['tsunami']==1)|(df['alert']=='red'), 
       ['alert', 'mag', 'magType', 'title', 'tsunami', 'type']]

In [91]:
data['mag'].mean()

np.float64(5.358032786885247)

In [92]:
data

,alert,mag,magType,title,tsunami,type
36,NaN,5.0,mww,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...",1,earthquake
118,green,6.7,mww,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,earthquake
501,green,5.6,mww,"M 5.6 - 128km SE of Kimbe, Papua New Guinea",1,earthquake
799,green,6.5,mww,"M 6.5 - 148km S of Severo-Kuril'sk, Russia",1,earthquake
816,green,6.2,mww,"M 6.2 - 94km SW of Kokopo, Papua New Guinea",1,earthquake
...,...,...,...,...,...,...
8561,NaN,5.4,mb,"M 5.4 - 228km S of Taron, Papua New Guinea",1,earthquake
8624,NaN,5.1,mb,"M 5.1 - 278km SE of Pondaguitan, Philippines",1,earthquake
9133,green,5.1,ml,"M 5.1 - 64km SSW of Kaktovik, Alaska",1,earthquake
9175,NaN,5.2,mb,"M 5.2 - 126km N of Dili, East Timor",1,earthquake


In [99]:
data[data['alert']=='green']

,alert,mag,magType,title,tsunami,type
118,green,6.70,mww,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,earthquake
501,green,5.60,mww,"M 5.6 - 128km SE of Kimbe, Papua New Guinea",1,earthquake
799,green,6.50,mww,"M 6.5 - 148km S of Severo-Kuril'sk, Russia",1,earthquake
816,green,6.20,mww,"M 6.2 - 94km SW of Kokopo, Papua New Guinea",1,earthquake
826,green,5.90,mww,"M 5.9 - 117km ESE of Kimbe, Papua New Guinea",1,earthquake
829,green,5.90,mww,"M 5.9 - 113km ESE of Kimbe, Papua New Guinea",1,earthquake
837,green,7.00,mww,"M 7.0 - 117km E of Kimbe, Papua New Guinea",1,earthquake
838,green,6.10,mb,"M 6.1 - 132km E of Kimbe, Papua New Guinea",1,earthquake
1015,green,5.00,ml,"M 5.0 - 61km SSW of Chignik Lake, Alaska",1,earthquake
1273,green,4.00,ml,"M 4.0 - 71km SW of Kaktovik, Alaska",1,earthquake


In [101]:
df.loc[
    (df.place.str.contains(r'AK$|Alaska$'))&(df.mag>4),
     ['alert', 'mag', 'magType', 'title', 'tsunami', 'type', 'place']
]

,alert,mag,magType,title,tsunami,type,place
304,NaN,4.7,mb,"M 4.7 - 71km NNE of Akutan, Alaska",0,earthquake,"71km NNE of Akutan, Alaska"
711,NaN,4.8,mb,"M 4.8 - 66km SSE of Nikolski, Alaska",0,earthquake,"66km SSE of Nikolski, Alaska"
1015,green,5.0,ml,"M 5.0 - 61km SSW of Chignik Lake, Alaska",1,earthquake,"61km SSW of Chignik Lake, Alaska"
1137,NaN,4.1,mb,"M 4.1 - 63km SE of Tanaga Volcano, Alaska",0,earthquake,"63km SE of Tanaga Volcano, Alaska"
2833,NaN,4.2,mb,"M 4.2 - 80km SE of Atka, Alaska",0,earthquake,"80km SE of Atka, Alaska"
3245,NaN,4.6,mb,"M 4.6 - 31km NNE of Nikolski, Alaska",0,earthquake,"31km NNE of Nikolski, Alaska"
4101,green,4.2,ml,"M 4.2 - 131km NNW of Arctic Village, Alaska",0,earthquake,"131km NNW of Arctic Village, Alaska"
4205,NaN,4.6,mb,"M 4.6 - 128km S of Shemya Island, Alaska",0,earthquake,"128km S of Shemya Island, Alaska"
4393,NaN,4.2,mb,"M 4.2 - 250km SE of Kodiak, Alaska",0,earthquake,"250km SE of Kodiak, Alaska"
4479,NaN,4.1,mb,"M 4.1 - 73km S of Amukta Island, Alaska",0,earthquake,"73km S of Amukta Island, Alaska"


In [102]:
df.loc[
    df.mag.between(6.5, 7.5),
     ['alert', 'mag', 'magType', 'title', 'tsunami', 'type', 'place']
    ]

,alert,mag,magType,title,tsunami,type,place
118,green,6.7,mww,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,earthquake,"262km NW of Ozernovskiy, Russia"
799,green,6.5,mww,"M 6.5 - 148km S of Severo-Kuril'sk, Russia",1,earthquake,"148km S of Severo-Kuril'sk, Russia"
837,green,7.0,mww,"M 7.0 - 117km E of Kimbe, Papua New Guinea",1,earthquake,"117km E of Kimbe, Papua New Guinea"
4363,green,6.7,mww,"M 6.7 - 263km NNE of Ndoi Island, Fiji",1,earthquake,"263km NNE of Ndoi Island, Fiji"
5263,red,7.5,mww,"M 7.5 - 78km N of Palu, Indonesia",1,earthquake,"78km N of Palu, Indonesia"


In [103]:
df.loc[
    df.magType.isin(['mw', 'mwb']),
     ['alert', 'mag', 'magType', 'title', 'tsunami', 'type', 'place']
    ]

,alert,mag,magType,title,tsunami,type,place
995,NaN,3.35,mw,"M 3.4 - 9km WNW of Cobb, CA",0,earthquake,"9km WNW of Cobb, CA"
1465,green,3.83,mw,"M 3.8 - 109km WNW of Trinidad, CA",0,earthquake,"109km WNW of Trinidad, CA"
2414,green,3.83,mw,"M 3.8 - 5km SW of Tres Pinos, CA",1,earthquake,"5km SW of Tres Pinos, CA"
4988,green,4.41,mw,"M 4.4 - 1km SE of Delta, B.C., MX",1,earthquake,"1km SE of Delta, B.C., MX"
6307,green,5.80,mwb,"M 5.8 - 297km NNE of Ndoi Island, Fiji",0,earthquake,"297km NNE of Ndoi Island, Fiji"
8257,green,5.70,mwb,"M 5.7 - 175km SSE of Lambasa, Fiji",0,earthquake,"175km SSE of Lambasa, Fiji"


In [108]:
[df.mag.idxmax(),
df.mag.idxmin()]

[5263, 2409]

In [112]:
df.loc[[df.mag.idxmax(), df.mag.idxmin()]]

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
5263,red,8.4,1000h3p4,https://earthquake.usgs.gov/fdsnws/event/1/que...,1.58900,18.0,27.0,",us1000h3p4,us1000h4p4,",7.50,mww,...,",us,us,",reviewed,1538128963480,"M 7.5 - 78km N of Palu, Indonesia",1,earthquake,",dyfi,finite-fault,general-text,geoserve,groun...",480.0,1539123134531,https://earthquake.usgs.gov/earthquakes/eventp...
2409,NaN,NaN,70806059,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.01871,NaN,123.0,",av70806059,",-1.26,ml,...,",av,",reviewed,1538746911930,"M -1.3 - 41km ENE of Adak, Alaska",0,earthquake,",geoserve,origin,phase-data,",-600.0,1538774135050,https://earthquake.usgs.gov/earthquakes/eventp...


In [117]:
df.loc[[df.mag.idxmax(), df.mag.idxmin()],['place']]


,place
5263,"78km N of Palu, Indonesia"
2409,"41km ENE of Adak, Alaska"


In [118]:
df.filter(items=['mag', 'place'])

,mag,place
0,1.35,"9km NE of Aguanga, CA"
1,1.29,"9km NE of Aguanga, CA"
2,3.42,"8km NE of Aguanga, CA"
3,0.44,"9km NE of Aguanga, CA"
4,2.16,"10km NW of Avenal, CA"
...,...,...
9327,0.62,"9km ENE of Mammoth Lakes, CA"
9328,1.00,"3km W of Julian, CA"
9329,2.40,"35km NNE of Hatillo, Puerto Rico"
9330,1.10,"9km NE of Aguanga, CA"


In [119]:
df.filter(like='mag')

,mag,magType
0,1.35,ml
1,1.29,ml
2,3.42,ml
3,0.44,ml
4,2.16,md
...,...,...
9327,0.62,md
9328,1.00,ml
9329,2.40,md
9330,1.10,ml


In [121]:
df.filter(regex=r'^t').head()

,time,title,tsunami,type,types,tz
0,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0
1,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0
2,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,earthquake,",dyfi,focal-mechanism,geoserve,nearby-cities,o...",-480.0
3,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0
4,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,scit...",-480.0


In [122]:
#

In [123]:
df = pd.read_csv("earthquakes.csv",
                 usecols=[
                     'alert',
                     'mag',
                     'magType',
                     'title',
                     'tsunami',
                     'type',
                     'place'
                 ])

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9332 entries, 0 to 9331
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alert    59 non-null     object 
 1   mag      9331 non-null   float64
 2   magType  9331 non-null   object 
 3   place    9332 non-null   object 
 4   title    9332 non-null   object 
 5   tsunami  9332 non-null   int64  
 6   type     9332 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 510.5+ KB


In [125]:
df.head()

,alert,mag,magType,place,title,tsunami,type
0,NaN,1.35,ml,"9km NE of Aguanga, CA","M 1.4 - 9km NE of Aguanga, CA",0,earthquake
1,NaN,1.29,ml,"9km NE of Aguanga, CA","M 1.3 - 9km NE of Aguanga, CA",0,earthquake
2,NaN,3.42,ml,"8km NE of Aguanga, CA","M 3.4 - 8km NE of Aguanga, CA",0,earthquake
3,NaN,0.44,ml,"9km NE of Aguanga, CA","M 0.4 - 9km NE of Aguanga, CA",0,earthquake
4,NaN,2.16,md,"10km NW of Avenal, CA","M 2.2 - 10km NW of Avenal, CA",0,earthquake


In [126]:
df.source="ozbekiston api"

In [127]:
df.head()

,alert,mag,magType,place,title,tsunami,type
0,NaN,1.35,ml,"9km NE of Aguanga, CA","M 1.4 - 9km NE of Aguanga, CA",0,earthquake
1,NaN,1.29,ml,"9km NE of Aguanga, CA","M 1.3 - 9km NE of Aguanga, CA",0,earthquake
2,NaN,3.42,ml,"8km NE of Aguanga, CA","M 3.4 - 8km NE of Aguanga, CA",0,earthquake
3,NaN,0.44,ml,"9km NE of Aguanga, CA","M 0.4 - 9km NE of Aguanga, CA",0,earthquake
4,NaN,2.16,md,"10km NW of Avenal, CA","M 2.2 - 10km NW of Avenal, CA",0,earthquake


In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9332 entries, 0 to 9331
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alert    59 non-null     object 
 1   mag      9331 non-null   float64
 2   magType  9331 non-null   object 
 3   place    9332 non-null   object 
 4   title    9332 non-null   object 
 5   tsunami  9332 non-null   int64  
 6   type     9332 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 510.5+ KB


In [129]:
df['source']="ozbekiston api"


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9332 entries, 0 to 9331
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alert    59 non-null     object 
 1   mag      9331 non-null   float64
 2   magType  9331 non-null   object 
 3   place    9332 non-null   object 
 4   title    9332 non-null   object 
 5   tsunami  9332 non-null   int64  
 6   type     9332 non-null   object 
 7   source   9332 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 583.4+ KB


In [131]:
df['mag_negative']= df.mag<0
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9332 entries, 0 to 9331
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   alert         59 non-null     object 
 1   mag           9331 non-null   float64
 2   magType       9331 non-null   object 
 3   place         9332 non-null   object 
 4   title         9332 non-null   object 
 5   tsunami       9332 non-null   int64  
 6   type          9332 non-null   object 
 7   source        9332 non-null   object 
 8   mag_negative  9332 non-null   bool   
dtypes: bool(1), float64(1), int64(1), object(6)
memory usage: 592.5+ KB


In [132]:
df.place

0                  9km NE of Aguanga, CA
1                  9km NE of Aguanga, CA
2                  8km NE of Aguanga, CA
3                  9km NE of Aguanga, CA
4                  10km NW of Avenal, CA
                      ...               
9327        9km ENE of Mammoth Lakes, CA
9328                 3km W of Julian, CA
9329    35km NNE of Hatillo, Puerto Rico
9330               9km NE of Aguanga, CA
9331               9km NE of Aguanga, CA
Name: place, Length: 9332, dtype: object

In [133]:
df.place.str.extract(r', (.*$)')[0].sort_values().unique()

array(['Afghanistan', 'Alaska', 'Argentina', 'Arizona', 'Arkansas',
       'Australia', 'Azerbaijan', 'B.C., MX', 'Barbuda', 'Bolivia',
       'Bonaire, Saint Eustatius and Saba ', 'British Virgin Islands',
       'Burma', 'CA', 'California', 'Canada', 'Chile', 'China',
       'Christmas Island', 'Colombia', 'Colorado', 'Costa Rica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'Ecuador region',
       'El Salvador', 'Fiji', 'Greece', 'Greenland', 'Guam', 'Guatemala',
       'Haiti', 'Hawaii', 'Honduras', 'Idaho', 'Illinois', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Italy', 'Jamaica', 'Japan', 'Kansas',
       'Kentucky', 'Kyrgyzstan', 'Martinique', 'Mauritius', 'Mayotte',
       'Mexico', 'Missouri', 'Montana', 'NV', 'Nevada', 'New Caledonia',
       'New Hampshire', 'New Mexico', 'New Zealand', 'Nicaragua',
       'North Carolina', 'Northern Mariana Islands', 'Oklahoma', 'Oregon',
       'Pakistan', 'Papua New Guinea', 'Peru', 'Philippines',
       'Puerto Rico', 'Roman

In [134]:
df.head()

,alert,mag,magType,place,title,tsunami,type,source,mag_negative
0,NaN,1.35,ml,"9km NE of Aguanga, CA","M 1.4 - 9km NE of Aguanga, CA",0,earthquake,ozbekiston api,False
1,NaN,1.29,ml,"9km NE of Aguanga, CA","M 1.3 - 9km NE of Aguanga, CA",0,earthquake,ozbekiston api,False
2,NaN,3.42,ml,"8km NE of Aguanga, CA","M 3.4 - 8km NE of Aguanga, CA",0,earthquake,ozbekiston api,False
3,NaN,0.44,ml,"9km NE of Aguanga, CA","M 0.4 - 9km NE of Aguanga, CA",0,earthquake,ozbekiston api,False
4,NaN,2.16,md,"10km NW of Avenal, CA","M 2.2 - 10km NW of Avenal, CA",0,earthquake,ozbekiston api,False


In [135]:
df['new_place']=df.place.str.replace(
    r'.* of ', '', regex=True
).str.replace(
    'the ', ''
).str.replace(
    r'CA$', 'California', regex=True
).str.replace(
    r'NV', 'Nevada', regex=True
).str.replace(
    r'MX$', 'Mexico', regex=True
).str.replace(
    r' region$', '', regex=True
).str.replace(
    'northern ', ''
).str.replace(
    'Fiji Islands', 'Fiji'
).str.replace(
    r'^.*, ', '', regex=True
).str.strip()

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9332 entries, 0 to 9331
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   alert         59 non-null     object 
 1   mag           9331 non-null   float64
 2   magType       9331 non-null   object 
 3   place         9332 non-null   object 
 4   title         9332 non-null   object 
 5   tsunami       9332 non-null   int64  
 6   type          9332 non-null   object 
 7   source        9332 non-null   object 
 8   mag_negative  9332 non-null   bool   
 9   new_place     9332 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(7)
memory usage: 665.4+ KB


In [137]:
df.head()

,alert,mag,magType,place,title,tsunami,type,source,mag_negative,new_place
0,NaN,1.35,ml,"9km NE of Aguanga, CA","M 1.4 - 9km NE of Aguanga, CA",0,earthquake,ozbekiston api,False,California
1,NaN,1.29,ml,"9km NE of Aguanga, CA","M 1.3 - 9km NE of Aguanga, CA",0,earthquake,ozbekiston api,False,California
2,NaN,3.42,ml,"8km NE of Aguanga, CA","M 3.4 - 8km NE of Aguanga, CA",0,earthquake,ozbekiston api,False,California
3,NaN,0.44,ml,"9km NE of Aguanga, CA","M 0.4 - 9km NE of Aguanga, CA",0,earthquake,ozbekiston api,False,California
4,NaN,2.16,md,"10km NW of Avenal, CA","M 2.2 - 10km NW of Avenal, CA",0,earthquake,ozbekiston api,False,California


In [148]:
df.assign(
    in_ca=df.new_place.str.endswith('California'),
    in_alaska=df.new_place.str.endswith('Alaska')
    ).sample(5, random_state=0)

,alert,mag,magType,place,title,tsunami,type,source,mag_negative,new_place,in_ca,in_alaska
7207,NaN,4.80,mwr,"73km SSW of Masachapa, Nicaragua","M 4.8 - 73km SSW of Masachapa, Nicaragua",0,earthquake,ozbekiston api,False,Nicaragua,False,False
4755,NaN,1.09,ml,"28km NNW of Packwood, Washington","M 1.1 - 28km NNW of Packwood, Washington",0,earthquake,ozbekiston api,False,Washington,False,False
4595,NaN,1.80,ml,"77km SSW of Kaktovik, Alaska","M 1.8 - 77km SSW of Kaktovik, Alaska",0,earthquake,ozbekiston api,False,Alaska,False,True
3566,NaN,1.50,ml,"102km NW of Arctic Village, Alaska","M 1.5 - 102km NW of Arctic Village, Alaska",0,earthquake,ozbekiston api,False,Alaska,False,True
2182,NaN,0.90,ml,"26km ENE of Pine Valley, CA","M 0.9 - 26km ENE of Pine Valley, CA",0,earthquake,ozbekiston api,False,California,True,False


In [150]:
df.assign(
    in_ca=df.new_place=='California',
    in_alaska=df.new_place=='Alasca',
    neither=lambda x: ~x.in_ca & ~x.in_alaska
).sample(5, random_state=0)

,alert,mag,magType,place,title,tsunami,type,source,mag_negative,new_place,in_ca,in_alaska,neither
7207,NaN,4.80,mwr,"73km SSW of Masachapa, Nicaragua","M 4.8 - 73km SSW of Masachapa, Nicaragua",0,earthquake,ozbekiston api,False,Nicaragua,False,False,True
4755,NaN,1.09,ml,"28km NNW of Packwood, Washington","M 1.1 - 28km NNW of Packwood, Washington",0,earthquake,ozbekiston api,False,Washington,False,False,True
4595,NaN,1.80,ml,"77km SSW of Kaktovik, Alaska","M 1.8 - 77km SSW of Kaktovik, Alaska",0,earthquake,ozbekiston api,False,Alaska,False,False,True
3566,NaN,1.50,ml,"102km NW of Arctic Village, Alaska","M 1.5 - 102km NW of Arctic Village, Alaska",0,earthquake,ozbekiston api,False,Alaska,False,False,True
2182,NaN,0.90,ml,"26km ENE of Pine Valley, CA","M 0.9 - 26km ENE of Pine Valley, CA",0,earthquake,ozbekiston api,False,California,True,False,False


In [151]:
df.drop(columns=['source'], inplace=True)

In [152]:
df.head()

,alert,mag,magType,place,title,tsunami,type,mag_negative,new_place
0,NaN,1.35,ml,"9km NE of Aguanga, CA","M 1.4 - 9km NE of Aguanga, CA",0,earthquake,False,California
1,NaN,1.29,ml,"9km NE of Aguanga, CA","M 1.3 - 9km NE of Aguanga, CA",0,earthquake,False,California
2,NaN,3.42,ml,"8km NE of Aguanga, CA","M 3.4 - 8km NE of Aguanga, CA",0,earthquake,False,California
3,NaN,0.44,ml,"9km NE of Aguanga, CA","M 0.4 - 9km NE of Aguanga, CA",0,earthquake,False,California
4,NaN,2.16,md,"10km NW of Avenal, CA","M 2.2 - 10km NW of Avenal, CA",0,earthquake,False,California


In [153]:
df.tail()

,alert,mag,magType,place,title,tsunami,type,mag_negative,new_place
9327,NaN,0.62,md,"9km ENE of Mammoth Lakes, CA","M 0.6 - 9km ENE of Mammoth Lakes, CA",0,earthquake,False,California
9328,NaN,1.00,ml,"3km W of Julian, CA","M 1.0 - 3km W of Julian, CA",0,earthquake,False,California
9329,NaN,2.40,md,"35km NNE of Hatillo, Puerto Rico","M 2.4 - 35km NNE of Hatillo, Puerto Rico",0,earthquake,False,Puerto Rico
9330,NaN,1.10,ml,"9km NE of Aguanga, CA","M 1.1 - 9km NE of Aguanga, CA",0,earthquake,False,California
9331,NaN,0.66,ml,"9km NE of Aguanga, CA","M 0.7 - 9km NE of Aguanga, CA",0,earthquake,False,California


In [154]:
df.tsunami.value_counts()

tsunami
0    9271
1      61
Name: count, dtype: int64

In [155]:
yes_tsunami=df[df['tsunami']==1]
no_tsunami=df[df['tsunami']==0]
[yes_tsunami.shape, no_tsunami.shape]

[(61, 9), (9271, 9)]

In [156]:
merged_data = pd.concat([yes_tsunami, no_tsunami])
merged_data

,alert,mag,magType,place,title,tsunami,type,mag_negative,new_place
36,NaN,5.00,mww,"165km NNW of Flying Fish Cove, Christmas Island","M 5.0 - 165km NNW of Flying Fish Cove, Christm...",1,earthquake,False,Christmas Island
118,green,6.70,mww,"262km NW of Ozernovskiy, Russia","M 6.7 - 262km NW of Ozernovskiy, Russia",1,earthquake,False,Russia
501,green,5.60,mww,"128km SE of Kimbe, Papua New Guinea","M 5.6 - 128km SE of Kimbe, Papua New Guinea",1,earthquake,False,Papua New Guinea
799,green,6.50,mww,"148km S of Severo-Kuril'sk, Russia","M 6.5 - 148km S of Severo-Kuril'sk, Russia",1,earthquake,False,Russia
816,green,6.20,mww,"94km SW of Kokopo, Papua New Guinea","M 6.2 - 94km SW of Kokopo, Papua New Guinea",1,earthquake,False,Papua New Guinea
...,...,...,...,...,...,...,...,...,...
9327,NaN,0.62,md,"9km ENE of Mammoth Lakes, CA","M 0.6 - 9km ENE of Mammoth Lakes, CA",0,earthquake,False,California
9328,NaN,1.00,ml,"3km W of Julian, CA","M 1.0 - 3km W of Julian, CA",0,earthquake,False,California
9329,NaN,2.40,md,"35km NNE of Hatillo, Puerto Rico","M 2.4 - 35km NNE of Hatillo, Puerto Rico",0,earthquake,False,Puerto Rico
9330,NaN,1.10,ml,"9km NE of Aguanga, CA","M 1.1 - 9km NE of Aguanga, CA",0,earthquake,False,California


In [157]:
additional_columns = pd.read_csv(
    'earthquakes.csv', usecols=['tz', 'felt', 'ids']
)
pd.concat([df.head(2), additional_columns.head(2)], axis=1)  # join = "inner"

,alert,mag,magType,place,title,tsunami,type,mag_negative,new_place,felt,ids,tz
0,NaN,1.35,ml,"9km NE of Aguanga, CA","M 1.4 - 9km NE of Aguanga, CA",0,earthquake,False,California,NaN,",ci37389218,",-480.0
1,NaN,1.29,ml,"9km NE of Aguanga, CA","M 1.3 - 9km NE of Aguanga, CA",0,earthquake,False,California,NaN,",ci37389202,",-480.0
